# Testando API do Trello

In [76]:
import requests
import json
import pandas as pd
from openpyxl import load_workbook

In [143]:
#Token para teste
#token_trello = '8b87e84eab71efca714b5c58277867922cd63a26087236eb3bc5d8db0e82dd28'

#Key para teste
#key_trello = 'fc23aaf60dd87002c17a29e2fc039d19'

#Endpoint para requests
#main_trello_endpoint = 'https://api.trello.com/1/'


## GET Request

### Getting jsonObj from board by id

In [144]:
#def getBoard(id):

#    url = main_trello_endpoint+'boards/'+id
#    params = {'key': key_trello, 'token': token_trello}
#    response = requests.get(url=url, params=params)
#    boardData = response.json()
#    print(boardData)


#getBoard('C65vXFhE')


## Creating Board Class

In [103]:
class Board:
    def __init__(self, key='fc23aaf60dd87002c17a29e2fc039d19', token='8b87e84eab71efca714b5c58277867922cd63a26087236eb3bc5d8db0e82dd28', main_endpoint='https://api.trello.com/1/', boardData = None, boardID = input("Insira o id presente no link do Quadro: "), boardLists = None, nome_da_lista = None) -> None:
        self.key = key
        self.token = token
        self.main_endpoint = main_endpoint
        self.boardId = boardID
        self.boardData = boardData
        self.boardLists = boardLists
        self.nome_da_lista = nome_da_lista
    
    def get_board(self):
        #Pega url base e acrescenta a parte do link faltante nesse caso "boards"
        url = self.main_endpoint +'boards/'+self.boardId
        params = {'key': self.key, 'token': self.token}
        response = requests.get(url=url, params=params)
        #Salva o jsonObj na variável boardData em forma de dicionário
        self.boardData = response.json()
        return self.boardData

    def get_board_name(self):
        #Verifica se as informações já foram puxadas do Trello
        if self.boardData == None:
            self.get_board()

        boardName = self.boardData['name']
        return(boardName)

    def get_board_id(self):
        #Verifica se as informações já foram puxadas do Trello
        if self.boardData == None:
            self.get_board()
        
        #pega o id do quadro
        boardIdLocal = self.boardData['id']

        return boardIdLocal

    def get_board_lists(self):
        #Acrescenta palavra ao link '/lists' 
        url = self.main_endpoint +'boards/'+self.boardId+'/lists'
        params = {'key': self.key, 'token': self.token}
        response = requests.get(url=url, params=params)
        self.boardLists = response.json()
        
        return self.boardLists

    def get_board_lists_names_and_idLists(self):
        self.get_board_lists()
        #Com certeza tem método mais eficiente -----------------------------------------------------------revisitar

        #adiciona nomes a lista nameList
        numberOfNames = len(self.boardLists)
        holder = None
        nameList = []  
        for i in range(numberOfNames):
            holder = dict(self.boardLists[i])
            nameList.append(holder['name'])

        #adiciona ids a lista idListList
        holder = None
        idListList = []
        for i in range(numberOfNames):
            holder = dict(self.boardLists[i])
            idListList.append(holder['id'])    
        
        #concatena nomes e ids no dicionario name_idList
        name_idList = dict(zip(nameList, idListList))

        return name_idList
        
    def get_idCards_on_board(self):
        self.get_board_lists_names_and_idLists()
        self.get_board_id()
        #altera link '/lists' para '/cards'
        url = self.main_endpoint +'boards/'+self.boardId+'/cards'
        params = {'key': self.key, 'token': self.token}
        response = requests.get(url=url, params=params)
        cards = response.json()
        
        #Faz uma lista com todos os ids dos cartoes
        numberOfCards = len(cards)
        holder = None
        cardsOnBoardIds = []
        for i in range(numberOfCards):
            holder = dict(cards[i])
            cardsOnBoardIds.append(holder['id'])  

         #Faz uma lista com todos os ids da lista que o cartao esta
        numberOfCards = len(cards)
        holder = None
        cardsOnBoardIdList = []
        for i in range(numberOfCards):
            holder = dict(cards[i])
            cardsOnBoardIdList.append(holder['idList'])
        
        #dicionario no formato {IdCartoes: IdLista que o cartao esta}
        CardsOnBoard_CardsOnBoardIdList = dict(zip(cardsOnBoardIds, cardsOnBoardIdList))

        return CardsOnBoard_CardsOnBoardIdList

    def get_a_card():
        pass


    def create_label(self):

        cor = input("Escolha a cor da tag(yellow, purple, blue, red, green, orange, black, sky, pink, lime): " )
        if cor == None:
            cor = null

        url = self.main_endpoint + '/labels?'
        
        params = {'idBoard': self.get_board_id(),'key': self.key, 'token': self.token}
        
        headersList = {
            "Accept": "application/json",
            "Content-Type": "application/json" 
            }

        #usando o openpyxl faz uma lista com os nomes das planilhas
        wb = load_workbook('trilha_do_conhecimento_-_sugest__o__1_.xlsx', read_only=True, keep_links=False)
        lista_de_planilhas = wb.sheetnames
        
        
        for i in range(len(lista_de_planilhas)):
        
            payload = json.dumps({
                "idBoard": self.get_board_id(),
                "name": str(lista_de_planilhas[i]),
                "color": cor,
                "limits": {}
                })
           
            response = requests.request("POST", url=url, params = params, data=payload,  headers=headersList)
            print(response)


    def get_labels(self):

        url = self.main_endpoint + 'boards/' + self.get_board_id() + '/labels'
        params = {'key': self.key, 'token': self.token}
        response = requests.request("GET", url=url, params=params)

        print(response.text)
    
            


    def create_card_on_a_list(self):
                
        #Pega o nome da lista que for inserida >>>sintaxe identica ao trello<<<
        self.nome_da_lista = input('Insira a lista desejada:')

        index_sheet = 0
        #Abre a planilha como data frame
        df = pd.read_excel('trilha_do_conhecimento_-_sugest__o__1_.xlsx')
        
       
        
        #Pega o nome dado no input salva o id referente a essa lista
        self.get_board_lists_names_and_idLists()
        lists_and_ids = self.get_board_lists_names_and_idLists()
        key_holder = lists_and_ids[self.nome_da_lista]
        
        #Request
        url = self.main_endpoint + '/cards?'

        params = {'idList': key_holder ,'key': self.key, 'token': self.token}

        headersList = {
            "Accept": "application/json",
            "Content-Type": "application/json" 
            }


         #Cria strings para a nomeação dos cards
        for i in range(len(df['Video'])):
            #Cria o titulo do card
            nome_p_criar_card = df['Video'][i] + ' - ' + df['Treinamento'][i] + ' (' + df['Intervalo video'][i] + ')'
    
            payload = json.dumps({  
                "name":str(nome_p_criar_card),   
                "desc": "",
                "idLabels":"63736124d2492f05dfd69cb3"})

            response = requests.request("POST", url=url, params = params, data=payload,  headers=headersList)

            print(response.text)



        

        #url = self.main_endpoint + '/cards?' + 'idList' + key_holder
        #params = {'key': self.key, 'token': self.token}
        #response = requests.post(url=url, params=params)

        #print(response)



        
        


#board id -> C65vXFhE
teste = Board()
#teste.create_card_on_backlog()
#teste.get_board_lists_names_and_idLists()
#teste.get_board_id()
#teste.create_label()
#teste.create_card_on_backlog()
teste.get_labels()
#teste.get_board_id()






[{"id":"63652598b7102e0179b83071","idBoard":"63652597b7102e0179b82c4c","name":"Fedex","color":"black"},{"id":"63652598b7102e0179b830ad","idBoard":"63652597b7102e0179b82c4c","name":"Vídeo","color":"blue"},{"id":"63652598b7102e0179b8308c","idBoard":"63652597b7102e0179b82c4c","name":"Freela","color":"sky"},{"id":"63652598b7102e0179b83062","idBoard":"63652597b7102e0179b82c4c","name":"PRIORIDADE","color":"red"},{"id":"63652598b7102e0179b83068","idBoard":"63652597b7102e0179b82c4c","name":"Machado Meyer","color":"pink"},{"id":"63652598b7102e0179b83080","idBoard":"63652597b7102e0179b82c4c","name":"Siemens  Healthineers","color":"black_dark"},{"id":"63652598b7102e0179b830bf","idBoard":"63652597b7102e0179b82c4c","name":"Wheaton","color":"green"},{"id":"63652598b7102e0179b8307d","idBoard":"63652597b7102e0179b82c4c","name":"Roteiro","color":"pink"},{"id":"63652598b7102e0179b8309b","idBoard":"63652597b7102e0179b82c4c","name":"Freela","color":"purple"},{"id":"63652598b7102e0179b830b9","idBoard":"636

# Baixar planilha e trabalhar df

In [85]:
#URL = "https://lhcv-my.sharepoint.com/:x:/g/personal/gupessanha_lhcv_onmicrosoft_com/Ea41ux0fJzlGrOC_HOtW3UwB9Fj4Vt_dB0gbaEJ_uG093w?e=kbFmeb"
#Faz download do arquivo da url
#response = requests.get(URL)
# abre a resposta em um outro arquivo chamado trilha_do_conhecimento_.........
#excel_file = open('trilha_do_conhecimento_-_sugest__o__1_.xlsx', "wb").write(response.content)


df = pd.read_excel('trilha_do_conhecimento_-_sugest__o__1_.xlsx', None)
#print(len(df))
#print(df)

#for i in range(len(df)):
#    df = pd.read_excel('trilha_do_conhecimento_-_sugest__o__1_.xlsx', i)
#    print(df)
#for i in range(len(df['Video'])):
#    nome_p_criar_card = df['Video'][i] + ' - ' + df['Treinamento'][i] + ' (' + df['Intervalo video'][i] + ')'
#    print(nome_p_criar_card)


wb = load_workbook('trilha_do_conhecimento_-_sugest__o__1_.xlsx', read_only=True, keep_links=False)
lista_de_planilhas = wb.sheetnames

for i in range(len(lista_de_planilhas)):
    print(lista_de_planilhas[i])



Sol 1.2 Cad de Prod Fornecedor
SUP 01 Cotação e pedido
SUP 3.2 Doc de entrada
SUP 04 PCS Solic de compras
SUP 06 Gestão de contratos
SUP 07 Gestão de compras
SUP 10 Aprovações gerais
Sol 1.3 Solic de pagamento
Controle de viagens
Sol 1.4 Solic de fundo fixo


# Criar etiquetas